In [1]:
from fastai.vision import *

In [2]:
import fastai

In [3]:
fastai.__version__

'1.0.54.dev0'

In [2]:
t1 = torch.randn(size=(1,1,128,128)).cuda()
t2 = torch.randn(size=(1,1,128,128,128)).cuda()

### a

In [55]:
conv2d = nn.Conv2d(1,1,3,bias=False,padding=1).cuda(); #conv2d.weight.data.normal_(5,1);
conv3d = nn.Conv3d(1,1,3,bias=False,padding=1).cuda(); #conv3d.weight.data.normal_(5,1);

In [63]:
a = t1.cuda()
for i in range(100):
    a = conv2d(a)
    if torch.isnan(a.mean()):break

In [64]:
i

99

In [65]:
a.mean(), a.std()

(tensor(-8.6458e-05, device='cuda:0', grad_fn=<MeanBackward1>),
 tensor(0.3323, device='cuda:0', grad_fn=<StdBackward0>))

In [70]:
a = t2.cuda()
for i in range(100):
    a = conv3d(a)
    if torch.isnan(a.mean()):break

In [72]:
i

299

In [73]:
a.mean(), a.std()

(tensor(5.8570e+13, device='cuda:0', grad_fn=<MeanBackward1>),
 tensor(inf, device='cuda:0', grad_fn=<StdBackward0>))

### b

In [7]:
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
loss = F.binary_cross_entropy_with_logits(input, target)

In [8]:
input

tensor([-1.4394,  0.2180, -0.9339], requires_grad=True)

In [9]:
target

tensor([0., 0., 1.])

### c

In [2]:
from data_utils import *

In [19]:
data_name='notl_ventricle_mr'
sample_size=10

In [20]:
f = data_dict[data_name]
train_paths, valid_paths, test1_paths, test2_paths = f()

In [21]:
if sample_size:
    idxs = np.random.choice(range(len(train_paths[0])), size=sample_size, replace=False)
    train_paths = np.array(train_paths)[:,idxs]

### d

In [4]:
from models import *

In [58]:
model = model_dict['residual_unet_wide']()

In [59]:
model

VolumetricResidualUnet(
  (down1): DownBlock(
    (down): Sequential(
      (0): Conv3d(1, 12, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): PReLU(num_parameters=1)
      (2): InstanceNorm3d(12, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (3): Dropout3d(p=0.2)
    )
    (preact_res): PreActBlock(
      (c1): Sequential(
        (0): InstanceNorm3d(12, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (1): PReLU(num_parameters=1)
        (2): Conv3d(12, 12, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): Dropout3d(p=0.2)
      )
      (c2): Sequential(
        (0): InstanceNorm3d(12, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (1): PReLU(num_parameters=1)
        (2): Conv3d(12, 12, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): Dropout3d(p=0.2)
      )
    )
  )
  (down2): DownBlock(
    (down): Sequential(
      (0): Conv3d(12, 2

In [60]:
tot=0
for p in model.parameters():
    tot+=p.data.numel()

In [61]:
tot

6028747

### e

In [77]:
data_name = 'notl_ventricle_mr'

In [78]:
# data
f = data_dict[data_name]
train_paths, valid_paths, test1_paths, test2_paths = f()
if sample_size:
    idxs = np.random.choice(range(len(train_paths[0])), size=sample_size, replace=False)
    train_paths = np.array(train_paths)[:,idxs]


train_ds = MRI_3D_Dataset(*train_paths)
valid_ds = MRI_3D_Dataset(*valid_paths)
test1_ds = MRI_3D_Dataset(*test1_paths) if test1_paths else None
test2_ds = MRI_3D_Dataset(*test2_paths) if test2_paths else None

data = DataBunch.create(train_ds=train_ds, valid_ds=valid_ds, bs=1)

In [79]:
xb, yb = data.one_batch()

In [80]:
yb.mean()

tensor(0.0020)